In [ ]:
"""
import packages
"""
import os, glob
import json
import pandas as pd
import numpy as np
import math, random
from datetime import datetime
import pickle
from matplotlib import pyplot as plt
import argparse
import path, sys, re, time
from collections import Counter
from scipy.spatial import distance_matrix
from scipy.signal import find_peaks

In [ ]:
"""
import custom packages
"""
from module_.info.testbed_info import d_files, g_files, c_files, p_files
from module_.info.config import config, feature_name
from module_.readText import create_episodes, time_correction
from module_.featureExtraction import feature_extraction
from module_.changePointDetection import change_point_detection
# from module_.evaluation import evaluation_
# from module_.analysis import neighbor_events as ne
# from module_.helper.labeling import feature_label

# Testbed (Seminar, multi-resident, episodes)

In [ ]:
"""
load raw files
"""
dir_="dataset/testbed/discrete/"
task_dict={
    0:  [np.load(dir_+file_name) for file_name in g_files.keys()],
    1:  [np.load(dir_+file_name) for file_name in d_files.keys()],
    2:  [np.load(dir_+file_name) for file_name in c_files.keys()],
    3:  [np.load(dir_+file_name) for file_name in p_files.keys()]
}
name_dict={
    0: 'G', 1: 'D', 2: 'C', 3: 'P'
}

episodes, trs, tags = create_episodes(task_dict, name_dict)
episodes=[time_correction(eps, trs[i]) for i, eps in enumerate(episodes)]

In [ ]:
"""make combinations
    1. pick one group type
    2. pick an activity stream from the group
    3. pick another group type w/o type 1.
    4. pick an activity stream from the group
"""
data_name='testbed'
metric='SEP'

for i, eps in enumerate(episodes):
    pairname=tags[i][0]+tags[i][2]
    print(i, tags[i])
    sensor_list=sorted(set(eps[:,0]))
    features=feature_extraction(eps, data_name, sensor_list)
    scores=change_point_detection(features, i, pairname, data_name=data_name, metric=metric, save=True)
    scores=np.array(scores)
    scores[scores<0]=0

    peak, _ =find_peaks(scores)
    positive=[i for i in peak if scores[i]>0.3]
    # positive=[i for i in range(len(scores)) if scores[i]>0.45]
    
###
    plt.title("{}-{}".format(tags[i], i))
    plt.ylabel('score')
    plt.xlabel('event')
    plt.ylim(0,0.7)
    plt.bar(range(len(eps)), scores)
    plt.plot(positive, np.array(scores)[positive], 'bx', label='peak')
    plt.axhline(y=0.45, linestyle=':', color='r', label='threshold')
    plt.axvline(x=trs[i], linestyle=':', color='g', label='transition')
    plt.legend()
    plt.savefig("./outputs/{}/{}/{}/{}/graph.png".format(data_name, metric, pairname, i))
    plt.clf()

###

In [12]:
"""
    hh101 Evaluation
    - load scores
"""
total_counts=np.zeros(4)
denom = numer = 0
for activity_folder in glob.glob("./outputs/{}/{}/*".format(data_name, metric)):
    # one type of pairs
    activity_pair=activity_folder.split("/")[-1]
    print(activity_pair)
    pair_counts=np.zeros(4) # TP, FP, TN, FN

    for episode_folder in glob.glob("{}/*".format(activity_folder)):
        denom+=1
        eps_order=int(episode_folder.split("/")[-1])
        eps, point=episodes[eps_order], trs[eps_order]
        scores=np.load("{}/scores.npy".format(episode_folder))

        peaks, _ = find_peaks(scores)
        positives=[i for i in peaks if scores[i]>0.1]
        numer+=len(positives)
        # positives=[i for i in range(len(scores)) if scores[i]>0.3]
        ttimestamp=float(eps[point][2])

        for i in range(len(scores)):
            if i in positives:
                if i==point:
                    pair_counts[0]+=1
                else:
                    timestamp_b=float(eps[i-1][2])
                    timestamp_a=float(eps[i][2])
                    if abs(ttimestamp-timestamp_b)<60 or abs(ttimestamp-timestamp_a)<60:
                        pair_counts[0]+=1
                    else:
                        pair_counts[1]+=1
            else:
                if i==point:
                    pair_counts[3]+=1
                else:
                    pair_counts[2]+=1
    TPR_=pair_counts[0]/(pair_counts[0]+pair_counts[3])
    FPR_=pair_counts[1]/(pair_counts[1]+pair_counts[2])
    print("Avg. TPR and FPR: ({}, {})".format(TPR_, FPR_))

    total_counts+=pair_counts

TPR=total_counts[0]/(total_counts[0]+total_counts[3])
FPR=total_counts[1]/(total_counts[1]+total_counts[2])
print("Total Avg. TPR and FPR: ({}, {})".format(TPR, FPR))


print(numer/denom)

GC
Avg. TPR and FPR: (0.6962025316455697, 0.13408827692703643)
GP
Avg. TPR and FPR: (0.7555555555555555, 0.13457249070631971)
PG
Avg. TPR and FPR: (0.7619047619047619, 0.13496780584447746)
PD
Avg. TPR and FPR: (0.82, 0.15209205020920502)
DG
Avg. TPR and FPR: (0.7530864197530864, 0.14532320105075522)
CD
Avg. TPR and FPR: (0.7843137254901961, 0.1522491349480969)
CG
Avg. TPR and FPR: (0.7951807228915663, 0.1317110853611433)
PC
Avg. TPR and FPR: (0.7821782178217822, 0.14619883040935672)
DC
Avg. TPR and FPR: (0.7352941176470589, 0.15955613014436543)
CP
Avg. TPR and FPR: (0.8037383177570093, 0.14167792285757166)
DP
Avg. TPR and FPR: (0.8118811881188119, 0.15587404540223873)
GD
Avg. TPR and FPR: (0.7619047619047619, 0.14067192192192193)
Total Avg. TPR and FPR: (0.775, 0.14465281784958645)
52.92666666666667
